# Water Quality Parameters Maps - Production

In [1]:
# Styling notebook

# System
import os
import sys

# Import scripts libraries for the project
sys.path.append('./src/python')

# Import the function to update the notebook style
from nbConfig import (css_styling)

css_styling()

## 0. Import WQP maps production library

In [2]:
import pandas as pd
import ipywidgets as wg
import wqpSNAPparams_S3 as wqpParams_S3
import wqpSNAPparams_L8 as wqpParams_L8
import wqpSNAPFunctions as wqpSNAP

In [3]:
SENSOR_LIST = ['S3','L8']
sensor = wg.Select(
    options = SENSOR_LIST,
    value = SENSOR_LIST[0],
    description = 'Select the sensor:',
    disabled = False,
    style = {
        'description_width': 'auto'
    },
    layout = wg.Layout(width='50%', height='100px'),
)
sensor

Select(description='Select the sensor:', layout=Layout(height='100px', width='50%'), options=('S3', 'L8'), sty…

In [12]:
in_path = f'./in/data/satellite_imagery/{sensor.value}'
out_path = f'./in/data/wqp/{sensor.value}'
cwd_path = wqpSNAP.inputParameters(in_path,out_path)

## 1. Define processing parameters

In [5]:
#Read the mean temperature file
df_t = pd.read_csv(os.path.join(cwd_path['in_parameters'],'mean_temp.csv'))
df_t_keys = list(df_t.keys())

In [6]:
df_t.head()

,Id Sensore,Data-Ora,Medio
0,14606,2019/01/01 00:00,5.5
1,14606,2019/01/01 00:10,5.5
2,14606,2019/01/01 00:20,5.4
3,14606,2019/01/01 00:30,5.3
4,14606,2019/01/01 00:40,5.3


In [17]:
# Read the atmospheric correction parameters file
df_atm = pd.read_csv(os.path.join(cwd_path['in_parameters'],'atm_corr.csv'))
df_atm_keys = list(df_t.keys())

In [18]:
df_atm['DateTime'] = pd.to_datetime(df_atm['DateTime'], format='%d/%m/%Y %H:%M')
df_atm['DateTime'] = df_atm['DateTime'].dt.date

In [19]:
df_atm.head()

,DateTime,Year,Month,Day,GMT Hour,Minute,t,Lu,Ld
0,2019-01-01,2019,1,1,10,10,0.93,0.47,0.80
1,2019-01-06,2019,1,6,10,10,0.93,0.46,0.79
2,2019-01-13,2019,1,13,10,10,0.91,0.56,0.96
3,2019-01-15,2019,1,15,10,10,0.95,0.32,0.55
4,2019-01-17,2019,1,17,10,10,0.87,0.80,1.34


In [20]:
# Temptative bounding box for the area of interest
bbox = {
    'minLat' : 45.3,
    'maxLat' : 46.65,
    'minLon' : 7.9,
    'maxLon' : 9.95,
}

In [21]:
# Format output bands
writeFormat = 'GeoTIFF'

## 2. WQP maps production

### S3 - Chlorophyll-a (CHL-a) and Total Suspended Matter (TSM)

In [ ]:
wqpParams = wqpParams_S3
for root, dirs, files in os.walk(in_path):
    for f in files:
        if f.endswith('.xml'):
            # 1. Read the product
            s3_image = wqpSNAP.snapProduct(os.path.join(root,f),bbox)
            s3_image.readSNAPProduct()
            s3_image.name = s3_image.path.split('\\')[-2].split('.')[0]
            print(s3_image.name)
            try:
                # 2. Update the bounding box for the subset selection
                params_subset = s3_image.updateSNAPSubset(wqpParams.params_subset)
                subset_product = wqpSNAP.executeSNAPFunction(s3_image.product, params_subset)
                # 3. Reproject the subset
                reproject_product = wqpSNAP.executeSNAPFunction(subset_product, wqpParams.params_reproject)
                # 4. Update the C2RCC temperature value. C2RCC wqp products
                params_C2RCC = s3_image.updateSNAPTemperature(df_t, wqpParams.params_C2RCC)
                c2rcc_product = wqpSNAP.executeSNAPFunction(reproject_product, params_C2RCC)
                # 5. Import vector layer
                importVector_product = wqpSNAP.executeSNAPFunction(c2rcc_product, wqpParams.params_importVector)
                # 6. Band Maths operations
                bandMaths_product_C2RCC = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths)
                bandMaths_product_oa = wqpSNAP.executeSNAPFunction(reproject_product, wqpParams.params_bandMaths_oa)
                bandMaths_product_rrs = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths_rrs)  
                bandMaths_product_masks = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths_masks)
                # 7. Extract bands
                # Product bands to be extracted
                bandExtract_product_chl = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl)
                bandExtract_product_tsm = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm)
                bandExtract_product_chl_no_clip = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl_no_clip)
                bandExtract_product_tsm_no_clip = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm_no_clip)
                bandExtract_product_chl_no_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl_no_masks)
                bandExtract_product_tsm_no_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm_no_masks)
                bandExtract_product_chl_cloud_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_chl_cloud_mask)
                bandExtract_product_tsm_cloud_mask = wqpSNAP.executeSNAPFunction(bandMaths_product_C2RCC, wqpParams.params_bandExtractor_tsm_cloud_mask)
                bandExtract_product_oa = wqpSNAP.executeSNAPFunction(bandMaths_product_oa, wqpParams.params_bandExtractor_oa)
                bandExtract_product_rrs = wqpSNAP.executeSNAPFunction(bandMaths_product_rrs, wqpParams.params_bandExtractor_rrs)
                bandExtract_product_masks = wqpSNAP.executeSNAPFunction(bandMaths_product_masks, wqpParams.params_bandExtractor_masks)
                # 8. Save bands
                # Define output paths
                sensor = s3_image.name.split('_')[0]
                sensor_date = s3_image.name.split('_')[8]
                out_path_chl = os.path.join(cwd_path['out_wqp']+'\\chl\\'+sensor+'_CHL_IT_'+sensor_date+'_L1')
                out_path_tsm = os.path.join(cwd_path['out_wqp']+'\\tsm\\'+sensor+'_TSM_IT_'+sensor_date+'_L1')
                out_path_chl_no_clip = os.path.join(cwd_path['out_wqp_no_clip']+'\\chl\\'+sensor+'_CHL_IT_'+sensor_date+'_L1')
                out_path_tsm_no_clip = os.path.join(cwd_path['out_wqp_no_clip']+'\\tsm\\'+sensor+'_TSM_IT_'+sensor_date+'_L1')
                out_path_chl_no_mask = os.path.join(cwd_path['out_wqp_no_mask']+'\\chl\\'+sensor+'_CHL_IT_'+sensor_date+'_L1')
                out_path_tsm_no_mask = os.path.join(cwd_path['out_wqp_no_mask']+'\\tsm\\'+sensor+'_TSM_IT_'+sensor_date+'_L1')
                out_path_chl_cloud_mask = os.path.join(cwd_path['out_wqp_cloud']+'\\chl\\'+sensor+'_CHL_IT_'+sensor_date+'_L1')
                out_path_tsm_cloud_mask = os.path.join(cwd_path['out_wqp_cloud']+'\\tsm\\'+sensor+'_TSM_IT_'+sensor_date+'_L1')
                out_path_mask = os.path.join(cwd_path['out_masks']+'\\'+sensor+'_IT_'+sensor_date+'_L1')
                out_path_oa = os.path.join(cwd_path['out_oa']+'\\'+sensor+'_IT_'+sensor_date+'_L1')
                out_path_rrs = os.path.join(cwd_path['out_rrs']+'\\'+sensor+'_IT_'+sensor_date+'_L1')
                # Save Bands
                wqpSNAP.exportProductBands(bandExtract_product_chl, out_path_chl, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_tsm, out_path_tsm, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_chl_no_clip, out_path_chl_no_clip, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_tsm_no_clip, out_path_tsm_no_clip, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_chl_no_mask, out_path_chl_no_mask, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_tsm_no_mask, out_path_tsm_no_mask, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_chl_cloud_mask, out_path_chl_cloud_mask, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_tsm_cloud_mask, out_path_tsm_cloud_mask, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_masks, out_path_mask, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_oa, out_path_oa, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_rrs, out_path_rrs, writeFormat)

                # Clean environment
                subset_product.dispose() 
                reproject_product.dispose()
                c2rcc_product.dispose()
                importVector_product.dispose()
                bandExtract_product_chl.dispose()
                bandExtract_product_tsm.dispose()
                bandExtract_product_chl_no_mask.dispose()
                bandExtract_product_tsm_no_mask.dispose()
                bandExtract_product_chl_cloud_mask.dispose()
                bandExtract_product_tsm_cloud_mask.dispose()
                bandExtract_product_rrs.dispose()
                bandExtract_product_oa.dispose()
                bandExtract_product_masks.dispose()
                del s3_image
                del subset_product
                del reproject_product
                del c2rcc_product
                del importVector_product
                del bandExtract_product_chl
                del bandExtract_product_tsm
                del bandExtract_product_chl_no_mask
                del bandExtract_product_tsm_no_mask

                del bandExtract_product_chl_cloud_mask
                del bandExtract_product_tsm_cloud_mask
                del bandExtract_product_rrs
                del bandExtract_product_oa
                del bandExtract_product_masks
            except:
                # Open a file with access mode 'a'
                file_object = open(os.path.join('./in/data/satellite_imagery',f'error_images{sensor.value}.txt'), 'a')
                # Append 'hello' at the end of file
                file_object.write(s3_image.name)
                file_object.write("\n")
                # Close the file
                file_object.close()

### L8 - Land Surface Water Temperature (LSWT)

In [22]:
wqpParams = wqpParams_L8
for root, dirs, files in os.walk(in_path):
    for f in files:
        if f.endswith('MTL.txt'):
            print(f)
            # 1. Read the product
            l8_image = wqpSNAP.snapProduct(os.path.join(root,f),bbox)
            l8_image.readSNAPProduct()
            l8_image.name = l8_image.path.split('\\')[-2].split('.')[0]
            params_bandMaths = l8_image.updateSNAPAtmCorr(df_atm,wqpParams.params_bandMaths)
            try:
                # 2. Update the bounding box for the subset selection
                params_subset = l8_image.updateSNAPSubset(wqpParams.params_subset)
                subset_product = wqpSNAP.executeSNAPFunction(l8_image.product, params_subset)
                # 3. Reproject the subset
    #                 reproject_product = wqpSNAP.executeSNAPFunction(subset_product, wqpParams.params_reproject) 
                # 4. Resample
                resample_product = wqpSNAP.executeSNAPFunction(subset_product, wqpParams.params_resample)
                # 5. Import-Vector
                importVector_product = wqpSNAP.executeSNAPFunction(resample_product, wqpParams.params_importVector)
                # 6. BandMaths
                bandMaths_product = wqpSNAP.executeSNAPFunction(importVector_product, params_bandMaths)
                bandMaths_product_masks = wqpSNAP.executeSNAPFunction(importVector_product, wqpParams.params_bandMaths_masks)
                # 7. BandExtract
                bandExtract_product_lswt = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_lswt)
                bandExtract_product_lswt_mid_high = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_lswt_mid_high)
                bandExtract_product_lswt_high = wqpSNAP.executeSNAPFunction(bandMaths_product, wqpParams.params_bandExtractor_lswt_high)
                bandExtract_product_masks = wqpSNAP.executeSNAPFunction(bandMaths_product_masks, wqpParams.params_bandExtractor_masks)
                # 8. Export bands
                # Output paths
                sensor_date = l8_image.name.split('_')[3]
                out_path_lswt = os.path.join(out_path,'wqp','lswt',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                out_path_lswt_mid_high = os.path.join(out_path,'wqp_mid_high_clouds','lswt',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                out_path_lswt_high = os.path.join(out_path,'wqp_high_clouds','lswt',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                out_path_mask = os.path.join(out_path,'masks',sensor.value+'_LSWT_IT_'+sensor_date+'_L1')
                #Save Bands
                wqpSNAP.exportProductBands(bandExtract_product_lswt, out_path_lswt, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_lswt_mid_high, out_path_lswt_mid_high, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_lswt_high, out_path_lswt_high, writeFormat)
                wqpSNAP.exportProductBands(bandExtract_product_masks, out_path_mask, writeFormat)
                
                subset_product.dispose() 
                importVector_product.dispose()
                resample_product.dispose()
                bandMaths_product.dispose()
                bandMaths_product_masks.dispose()
                bandExtract_product_lswt.dispose()
                bandExtract_product_lswt_mid_high.dispose()
                bandExtract_product_lswt_high.dispose()
                bandExtract_product_masks.dispose()
#                 del l8_image
                del subset_product
                del importVector_product
                del resample_product
                del importVector_product
                del bandExtract_product_lswt
                del bandExtract_product_lswt_mid_high
                del bandExtract_product_lswt_high
                del bandExtract_product_masks
                del bandMaths_product
                del bandMaths_product_masks
            except:
                # Open a file with access mode 'a'
                file_object = open(os.path.join('./in/data/satellite_imagery',f'error_images_{sensor.value}.txt'), 'a')
                # Append 'hello' at the end of file
                file_object.write(l8_image.name)
                file_object.write("\n")
                # Close the file
                file_object.close()
print('THE END')

LC08_L1TP_193028_20220429_20220429_02_RT_MTL.txt
Date: 2022-04-29 00:00:00 
 Lu: 1.06, t: 0.85, Ld: 1.76
LC08_L1TP_193028_20220515_20220519_02_T1_MTL.txt
Date: 2022-05-15 00:00:00 
 Lu: 4.31, t: 2.74, Ld: 0.64
LC08_L1TP_193028_20220531_20220531_02_RT_MTL.txt
Date: 2022-05-31 00:00:00 
 Lu: 3.03, t: 1.86, Ld: 0.74
LC08_L1TP_194028_20220114_20220123_02_T1_MTL.txt
Date: 2022-01-14 00:00:00 
 Lu: 0.27, t: 0.96, Ld: 0.47
LC08_L1TP_194028_20220130_20220130_02_RT_MTL.txt
Date: 2022-01-30 00:00:00 
 Lu: 0.54, t: 0.92, Ld: 0.92
LC08_L1TP_194028_20220130_20220204_02_T1_MTL.txt
Date: 2022-01-30 00:00:00 
 Lu: 0.54, t: 0.92, Ld: 0.92
LC08_L1TP_194028_20220215_20220223_02_T1_MTL.txt
Date: 2022-02-15 00:00:00 
 Lu: 0.61, t: 0.9, Ld: 1.03
LC08_L1TP_194028_20220303_20220309_02_T1_MTL.txt
Date: 2022-03-03 00:00:00 
 Lu: 0.46, t: 0.93, Ld: 0.79
LC08_L1TP_194028_20220319_20220329_02_T1_MTL.txt
Date: 2022-03-19 00:00:00 
 Lu: 0.48, t: 0.92, Ld: 0.82
LC08_L1TP_194028_20220404_20220412_02_T1_MTL.txt
Date: 2

UnboundLocalError: local variable 'bExpression' referenced before assignment

In [ ]:
# List bands and masks available for the opened product
# list(l8_image.product.getBandNames())
# list(l8_image.product.getMaskGroup().getNodeNames())

In [ ]:
a = wqpSNAP.extractAtmCorr(l8_image.name, df_atm)